### Prendi sempre il dataset fetch_20newsgroups. Questa volta però filtra per le news a tema medico (utilizza la categoria sci.med)
- Crea un corpus prendendo tutte le frasi delle prime 100 news
- Fai preprocessing come nel notebook precedente per generare dei 4-Grammi
- Non considerare come vocabolario tutte le parole presenti nelle news considerate, ma prova a tenere solamente le 3000 più presenti
- Calcola la matrice di probabilità per solamente per gli n-grammi presenti nel corpus
- Crea un Language Model come nel notebook precedente e genera del testo
- Prova a generare del testo sia non facendo random sampling sia facendolo
- Prova a variare la soglia di threshold (con range tra 0.0000001 e 0.0003) sopra la quale tenere i token da generare per vedere (anche con un piccolo esempio) come cambiano i testi generati dal language model



<font color='red'>Va BENISSIMO fare copia-incolla dai notebook precedenti: <br>
&nbsp;&nbsp;&nbsp;&nbsp; → non devi imparare cose a memoria, devi capire come funzionano e sapere dove copiare, cosa modifiare e come! <br>
 Riesci a superare tutti gli errori ed a generare del testo a tema medico?</font>

In [ ]:
from sklearn.datasets import fetch_20newsgroups         # Dataset che contiene testo di articoli di giornale appartenenti a 20 categorie differenti
import nltk                                             # NLP toolkit
import re                                               # Libreria per operazioni con le espressioni regolari
from collections import defaultdict, Counter
import numpy as np
import pandas as pd
import random
import itertools

nltk.download('punkt')                                  # Con questo comando si scarica il tokenizzatore 'Punkt'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Scarichiamo il nostro dataset

In [ ]:
train_news_texts, category = fetch_20newsgroups(subset="train", categories=["sci.med"], return_X_y=True, remove=['headers', 'footers', 'quotes'])

## Definiamo la funzione creata in precedenza per processare il dataset e creare gli N-Grammi

In [ ]:
def preprocessing(text):
    """
    Funzione che preprocessa il testo per creare una frase tokenizzata.

    Args:
        text: stringa contenente il testo da preprocessare e tokenizzare

    Returns:
        testo preprocessato e tokenizzato
    """
    text = text.lower()
    text = text.replace('\n', ' ')
    text = re.sub(r'[^a-zA-Z0-9.?! ]+', '', text)
    text = re.sub(' +', ' ', text)
    text = text.strip()
    text_tokenized = nltk.word_tokenize(text)
    return text_tokenized


In [ ]:
def sentence_to_ngram(tokenized_sentence, n=3):
    """
    Funzione che restituisce tutti i n-grammi contenuti all'interno della frase tokenizzata.

    Args:
        tokenized_sentence: lista di parole/tokens che compongono la frase
        n: n-grammi da considerare

    Returns:
        lista di tutti i n-grammi presenti all'interno della frase tokenizzata
    """

    ngram_list = []
    tokenized_sentence = ['<s>'] * (n - 1) + tokenized_sentence + ['</s>']
    for i in range(len(tokenized_sentence) - n + 1):
        # the sliding window starts at position i and contains 3 words
        ngram = tokenized_sentence[i : i + n]
        ngram_list.append(ngram)
    return ngram_list



# Prendi il sample di testi che useremo come base per costruire il Language Model



In [ ]:
train_news_texts_sample = [sentence for x in train_news_texts[:100] for sentence in nltk.sent_tokenize(x.strip())]
for text in train_news_texts_sample[:10]:
  print(text)
  print("-"*80)


[reply to keith@actrix.gen.nz (Keith Stewart)]
 
 
It would help if you (and anyone else asking for medical information on
some subject) could ask specific questions, as no one is likely to type
in a textbook chapter covering all aspects of the subject.
--------------------------------------------------------------------------------
If you are
looking for a comprehensive review, ask your local hospital librarian.
--------------------------------------------------------------------------------
Most are happy to help with a request of this sort.
--------------------------------------------------------------------------------
Briefly, this is a condition in which patients who have significant
residual weakness from childhood polio notice progression of the
weakness as they get older.
--------------------------------------------------------------------------------
One theory is that the remaining motor
neurons have to work harder and so die sooner.
-----------------------------------------

# Definisci il vocabolario tenendo solamente le prime 3000 parole più comuni

In [ ]:
train_news_texts_sample_processed = [preprocessing(text) for text in train_news_texts_sample]

In [ ]:
M = 3000
corpus = [x for y in train_news_texts_sample_processed for x in y]
word_count = Counter(corpus)
vocabulary = Counter(word_count).most_common(M)
vocabulary = [word for word, frequency in vocabulary]

In [ ]:
# Siccome abbiamo delle parole non presenti nel nostro vocabolario, aggiungiamoci anche il token UNK e i token di inizio/fine frase
vocabulary.append('UNK')
vocabulary.append('<s>')
vocabulary.append('</s>')

In [ ]:
train_news_texts_sample_processed_unk = []
for sentence in train_news_texts_sample_processed:
  new_sentence = []
  for word in sentence:
    if word not in vocabulary:
      new_sentence.append('UNK')
    else:
      new_sentence.append(word)
  train_news_texts_sample_processed_unk.append(new_sentence)

In [ ]:
n = 4
quadrigrams_list = [sentence_to_ngram(tokenized_sentence, n) for tokenized_sentence in train_news_texts_sample_processed_unk]
quadrigrams_list = [tuple(x) for y in quadrigrams_list for x in y]
count_quadrigrams = Counter(quadrigrams_list)
count_quadrigrams.most_common(20)

[(('<s>', '<s>', '<s>', 'UNK'), 129),
 (('<s>', '<s>', '<s>', 'the'), 121),
 (('<s>', '<s>', '<s>', 'i'), 86),
 (('UNK', 'UNK', '.', '</s>'), 50),
 (('<s>', '<s>', '<s>', 'this'), 39),
 (('<s>', '<s>', 'UNK', 'UNK'), 36),
 (('<s>', '<s>', '<s>', 'if'), 35),
 (('<s>', '<s>', '<s>', 'it'), 34),
 (('<s>', '<s>', '<s>', 'in'), 31),
 (('<s>', '<s>', '<s>', 'a'), 26),
 (('volume', '6', 'number', '10'), 25),
 (('6', 'number', '10', 'april'), 25),
 (('number', '10', 'april', '20'), 25),
 (('10', 'april', '20', '1993'), 25),
 (('hicnet', 'medical', 'newsletter', 'page'), 25),
 (('UNK', 'UNK', 'UNK', 'UNK'), 22),
 (('<s>', '<s>', '<s>', 'what'), 19),
 (('<s>', '<s>', '<s>', 'but'), 16),
 (('<s>', '<s>', 'if', 'you'), 15),
 (('<s>', '<s>', '<s>', 'and'), 15)]

# Crea la matrice dei conteggi

In [ ]:
def single_pass_ngram_count_matrix(count_ngrams):
    """
    Crea la matrice dei conteggi di tri-grammi utilizzando il corpus passato in input.

    Args:
        count_ngrams: Conteggio dei n-grammi presenti nel corpus

    Returns:
        n_minus_one_grams: lista di tutti i bigrammi, utilizzato come indice di riga della matrice
        vocabulary: lista di tutte le parole presenti nel corpus, utilizzato come indice di colonna
        count_matrix: pandas dataframe con i bigrammi prefixes come righe,
                      le parole del vocabolario come colonne
                      e il conteggio delle combinazioni bigramma/parola come valore
    """
    n_minus_one_grams = []
    vocabulary = []
    count_matrix_dict = defaultdict(dict)

    # go through the corpus once with a sliding window
    for ngram, count in count_ngrams.items():

        n_minus_one_gram = ngram[0 : -1]
        if not n_minus_one_gram in n_minus_one_grams:
            n_minus_one_grams.append(n_minus_one_gram)

        last_word = ngram[-1]
        if not last_word in vocabulary:
            vocabulary.append(last_word)

        if (n_minus_one_gram,last_word) not in count_matrix_dict:
            count_matrix_dict[n_minus_one_gram,last_word] = count

    # convert the count_matrix to np.array to fill in the blanks
    count_matrix = np.zeros((len(n_minus_one_grams), len(vocabulary)))
    for ngram_key, ngram_count in count_matrix_dict.items():
        count_matrix[n_minus_one_grams.index(ngram_key[0]), \
                     vocabulary.index(ngram_key[1])]\
        = ngram_count

    # np.array to pandas dataframe conversion
    count_matrix = pd.DataFrame(count_matrix, index=n_minus_one_grams, columns=vocabulary)
    return n_minus_one_grams, vocabulary, count_matrix


In [ ]:
n_minus_one_grams, vocabulary, count_matrix = single_pass_ngram_count_matrix(count_quadrigrams)

print(count_matrix.iloc[:5, :5])

                                 reply   to  keithactrix.gen.nz  keith  \
(<s>, <s>, <s>)                    1.0  7.0                 0.0    0.0   
(<s>, <s>, reply)                  0.0  1.0                 0.0    0.0   
(<s>, reply, to)                   0.0  0.0                 1.0    0.0   
(reply, to, keithactrix.gen.nz)    0.0  0.0                 0.0    1.0   
(to, keithactrix.gen.nz, keith)    0.0  0.0                 0.0    0.0   

                                 stewart  
(<s>, <s>, <s>)                      0.0  
(<s>, <s>, reply)                    0.0  
(<s>, reply, to)                     0.0  
(reply, to, keithactrix.gen.nz)      0.0  
(to, keithactrix.gen.nz, keith)      1.0  


In [ ]:
# create the probability matrix from the count matrix
row_sums = count_matrix.sum(axis=1)
# delete each row by its sum
prob_matrix = count_matrix.div(row_sums, axis=0)

print(prob_matrix.iloc[:5, :5])


                                    reply        to  keithactrix.gen.nz  \
(<s>, <s>, <s>)                  0.000729  0.005102                 0.0   
(<s>, <s>, reply)                0.000000  1.000000                 0.0   
(<s>, reply, to)                 0.000000  0.000000                 1.0   
(reply, to, keithactrix.gen.nz)  0.000000  0.000000                 0.0   
(to, keithactrix.gen.nz, keith)  0.000000  0.000000                 0.0   

                                 keith  stewart  
(<s>, <s>, <s>)                    0.0      0.0  
(<s>, <s>, reply)                  0.0      0.0  
(<s>, reply, to)                   0.0      0.0  
(reply, to, keithactrix.gen.nz)    1.0      0.0  
(to, keithactrix.gen.nz, keith)    0.0      1.0  


In [ ]:
print(prob_matrix.max().max())

1.0


## Generiamo del testo utilizzando il nostro language model

In [ ]:
def generate_text(n: int, prob_matrix: pd.DataFrame,
                  token_count: int, threshold_prob = 0.005, random_sampling: bool = False):
    """
    Funzione per generare del testo partendo dalla matrice di probabilità.

    Args:
        n: modello n-gramma da utilizzare
        prob_matrix: matrice di probabilità
        token_count: numero di token da generare
        threshold_prob: soglia di probabilità sopra la quale considerare i token
        random_sampling: booleano che dice se effettuare un sampling tra i token a probabilità non nulla oppure prendere sempre quello a probabilità massima

    Returns:
        bigrams: lista di tutti i bigrammi, utilizzato come indice di riga della matrice
        vocabulary: lista di tutte le parole presenti nel corpus, utilizzato come indice di colonna
        count_matrix: pandas dataframe con i bigrammi prefixes come righe,
                      le parole del vocabolario come colonne
                      e il conteggio delle combinazioni bigramma/parola come valore
    """
    context_queue = (n - 1) * ['<s>']
    result = []
    for _ in range(token_count):
      if random_sampling:
        if tuple(context_queue) in prob_matrix.index.tolist():
          nonzero_probs = (prob_matrix.loc[[tuple(context_queue)]]> threshold_prob).any()
          tokens_list = nonzero_probs.index[nonzero_probs].tolist()
          obj = random.sample(tokens_list, 1)[0]
        else:
          return ' '.join(result)
      else:
        obj = prob_matrix.loc[[tuple(context_queue)]].max().idxmax()
      result.append(obj)
      if n > 1:
          context_queue.pop(0)
          if obj == '.':
              context_queue = (n - 1) * ['<s>']
          else:
              context_queue.append(obj)
    return ' '.join(result)


In [ ]:
for i in range(10):
  print(generate_text(n, prob_matrix, 20, random_sampling=False))

UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK
UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK
UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK
UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK
UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK
UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK
UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK
UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK
UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK
UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK . UNK UNK


In [ ]:
for i in range(10):
  print(generate_text(n, prob_matrix, 20, random_sampling=True, threshold_prob = 0.0000001))

melittin </s>
which practitioner is more likely to get UNK in the conference center is strictly limited so UNK will be examined
obviously shell need a UNK to get anything and then i inject at about a 45 degree UNK it still
100 yrs ago it was done all the time at UNK but we are adding new information . tissue UNK
however its results were questionable at best since the researchers did not go on to measure whether the increase in
do some of you get a UNK UNK or internet UNK anonymous ftp UNK also UNK into windows UNK .
thats quite UNK since the UNK UNK for more than 30 years and finding it UNK to all UNK and
melittin </s>
nearly all hivinfected children UNK 47 new evidence that the hiv itself caused the skin disease . perhaps such pictures
those mice however became sick and died too soon after birth to study in depth . below is a press


In [ ]:
for i in range(10):
  print(generate_text(n, prob_matrix, 20, random_sampling=True, threshold_prob = 0.00005))

diseases such as aids . clinical research news high UNK assisted UNK technologies ......................... 24 4 . each child will
experiment and empirical studies are an important part of true scientific methodology is scientific thinking . profits of the group
glutamic acid is the most UNK UNK of UNK patients met the case definition of UNK e. coli and the
articles low levels UNK UNK linked to serious asthma attacks UNK 29 nih consensus development conference on UNK ..................... 31
free demo 5 outside us . despite the UNK findings scientists are UNK whether the same result occurs in the
how long ago was this ? </s>
younger age groups and the UNK of eligibility a couple of liters of corn much of it intact kernals .
hrsa is one of the a UNK . jb 2 were you able to understand my answer to your question
each child will receive six UNK every four weeks for six UNK be UNK for when necessary with glasses or
before that i used to get bloody noses nighttime asthma attacks and eyes so 

In [ ]:
for i in range(10):
  print(generate_text(n, prob_matrix, 20, random_sampling=True, threshold_prob = 0.00015))

one or more of the other UNK made by genentech inc. of south san francisco and the other by biocine
div of field epidemiology epidemiology program office UNK diseases UNK div of health nevada state dept of human resources .
rutin is a UNK UNK in clinical trials ? </s>
when initially UNK up the software it lets you select several options 1 text graphics 2 UNK source ie scanner
later david </s>
worked UNK on windows UNK though it did occasionally UNK with a prize cure . even after i try to
shavlik 6 comparison of two approaches to the prediction of protein UNK k. UNK k. asai m. ishikawa a. UNK
eat a healthy balanced diet and relax . preliminary report foodborne outbreak of escherichia coli o157h7 other UNK e. coli
studies of tissue taken from the wartlike skin tumors showed that they were a type of UNK he is having
free demo 5 outside us . elevated levels of glu and UNK in six western states . having had limited


In [ ]:
for i in range(10):
  print(generate_text(n, prob_matrix, 20, random_sampling=True, threshold_prob = 0.002))

it UNK my family no end since im at an UNK school sort of validated it for them ... then
one artificial sweetener that is not typically UNK of causing cancer is aspartame . 2 . jb 3 ron ...
good story UNK but it wont UNK . thank you UNK live from new york its saturday night ... </s>
this led them to UNK that the root cause of allergy may have a psychological UNK or basis . is
as growing numbers of women of UNK age become exposed to hiv through injection drug use or infected sexual partners
are there noninvasive ways of diagnosing a hernia ? </s>
UNK patrick ph.d. communications research UNK UNK canada UNK </s>
from article UNK by UNK the areas that are least likely to hurt are where you have a little gut
am j public health UNK . ive tried this when UNK and UNK ! </s>
so why bother with it ? </s>


In [ ]:
for i in range(10):
  print(generate_text(n, prob_matrix, 20, random_sampling=True, threshold_prob = 0.03))

the UNK food UNK practice is to cook ground beef thoroughly until the interior is no longer UNK and the
the only problem is the personal dictionary will only handle about 200 words . UNK . UNK UNK UNK .
UNK UNK UNK UNK UNK and the food and drug administration have given aspartame one of the researchers who conducted
i am especially interested in getting reviews of these products from people who have UNK them or are using them
the UNK of certain health problems but also to be UNK to most calendar programs . the UNK of science
the only fungus i know of from california is UNK . UNK . i think 1 outlaw the use of
UNK UNK UNK UNK UNK and functional UNK . the only limits i know of from california is UNK .
the UNK who are UNK by drug resistance in conditions such as gonorrhea malaria pneumococcal disease UNK UNK tb and
i would love to have anyone come up with a study to support their claims that the placebo effect from
UNK UNK UNK and put it over the spot . i dont want to go into a lot of mone